# Process DeepSea datset

In this notebook, the DeepSea dataset is acquired and parsed to generate a smaller transcription factor dataset, consisting of CTCF, GABP, SP1, SRF, and YY1, for K562 and HepG2 celltypes. The dataset is first downloaded directly from DeepSea webserver and then custom scripts convert these into a h5py file.

In [1]:
import os, sys, h5py, scipy.io
import numpy as np
import subprocess as sp

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# download DeepSea dataset

In [3]:
# download deepsea dataset into data folder, if it does not exist
if not os.path.isdir('../data/deepsea_train'):
    print('downloading DeepSea dataset')
    os.system('wget http://deepsea.princeton.edu/media/code/deepsea_train_bundle.v0.9.tar.gz -O ../data/deepsea_train_bundle.v0.9.tar.gz') 
    print('decompressing DeepSea dataset')
    os.system('tar xzvf ../data/deepsea_train_bundle.v0.9.tar.gz -C ../data ')
    os.system('rm ../data/deepsea_train_bundle.v0.9.tar.gz')

# define functions

In [4]:
def load_DeepSea_subset(filepath, class_range=range(918)):
    """ function to load DeepSea's dataset of specific transcription factors specified 
        by class_range. The output is a h5py file with the sequences represented
        as a 4D tensor for input into Lasagne/Theano convolution layers.  The labels
        is a 2D matrix where each row corresponds to a new sequence. """
    
    def data_subset(y, class_range):
        " gets a subset of data in the class_range"
        data_index = []
        for i in class_range:
            index = np.where(y[:, i] == 1)[0]
            data_index = np.concatenate((data_index, index), axis=0)
        unique_index = np.unique(data_index)
        return unique_index.astype(int)

    print "loading training data"
    trainmat = h5py.File(os.path.join(filepath,'train.mat'), 'r')
    y_train = np.transpose(trainmat['traindata'], axes=(1,0))
    index = data_subset(y_train, class_range)
    y_train = y_train[:,class_range]
    y_train = y_train[index,:]
    X_train = np.transpose(trainmat['trainxdata'], axes=(2,1,0)) 
    X_train = X_train[index,:,:]
    X_train = X_train[:,[0,2,1,3],:]
    X_train = np.expand_dims(X_train, axis=3)
    train = (X_train.astype(np.int8), y_train.astype(np.int8))
    
    print "loading validation data"  
    validmat = scipy.io.loadmat(os.path.join(filepath,'valid.mat'))
    y_valid = np.array(validmat['validdata'])
    index = data_subset(y_valid,class_range)
    y_valid = y_valid[:, class_range]
    y_valid = y_valid[index,:]
    X_valid = np.transpose(validmat['validxdata'], axes=(0,1,2))  
    X_valid = X_valid[index,:,:]
    X_valid = X_valid[:,[0,2,1,3],:]
    X_valid = np.expand_dims(X_valid, axis=3)
    valid = (X_valid.astype(np.int8), y_valid.astype(np.int8))
    
    print "loading test data"
    testmat = scipy.io.loadmat(os.path.join(filepath,'test.mat'))
    y_test = np.array(testmat['testdata'])
    index = data_subset(y_test,class_range)
    y_test = y_test[:, class_range]
    y_test = y_test[index,:]
    X_test = np.transpose(testmat['testxdata'], axes=(0,1,2)) 
    X_test = X_test[index,:,:]
    X_test = X_test[:,[0,2,1,3],:]
    X_test = np.expand_dims(X_test, axis=3)
    test = (X_test.astype(np.int8), y_test.astype(np.int8))

    return train, valid, test 

def process_DeepSea_subset(train, valid, valid_percentage=0.1):
    """merge training and validation data, shuffle, and reallocate 
       based on 90% training and 10% cross-validation """
    
    X_train = np.vstack([train[0], valid[0]])
    Y_train = np.vstack([train[1], valid[1]])
    index = np.random.permutation(X_train.shape[0])
    cutoff = np.round(X_train.shape[0]*valid_percentage).astype(int)
    valid = (X_train[:cutoff], Y_train[:cutoff])
    train = (X_train[cutoff:], Y_train[cutoff:])
    
    return train, valid


def save_DeepSea_subset(grp, train, valid, test):
    """ save to h5py dataset """
    print "saving datset"
    X_train = grp.create_dataset('X_train', data=train[0], dtype='int8', compression="gzip")
    Y_train = grp.create_dataset('Y_train', data=train[1], dtype='int8', compression="gzip")
    X_valid = grp.create_dataset('X_valid', data=valid[0], dtype='int8', compression="gzip")
    Y_valid = grp.create_dataset('Y_valid', data=valid[1], dtype='int8', compression="gzip")
    X_test = grp.create_dataset('X_test', data=test[0], dtype='int8', compression="gzip")
    Y_test = grp.create_dataset('Y_test', data=test[1], dtype='int8', compression="gzip")


# parse subset of DeepSea dataset

In [5]:
core_names = ['Arid3a', 'CEBPB', 'FOSL1', 'Gabpa', 'MAFK', 'MAX', 
              'MEF2A', 'NFYB', 'SP1', 'SRF', 'STAT1', 'YY1']
core_index = [592, 602, 344, 345, 635, 636, 349, 642, 359, 361, 661, 369]
#core_index =  [547, 602, 344, 345, 635, 636, 218, 642, 237, 238, 535, 369]

# save datasets in a hdf5 file under groups HepG2 and K562
data_path = '../data/deepsea_train/'

# load deep sea dataset
train, valid, test = load_DeepSea_subset(data_path, class_range=core_index)

loading training data
loading validation data
loading test data


In [6]:
print("number of training samples for each class")
np.sum(train[1], axis=0)

number of training samples for each class


array([27652, 85354, 19724, 34194, 43528, 87290,  9792, 22758, 17450,
        7528,  4516, 31146])

# save dataset

In [7]:
#train, valid = process_DeepSea_subset(train, valid, valid_percentage=0.1)        
with h5py.File('../data/invivo_dataset.h5', 'w') as fout:
    X_train = fout.create_dataset('X_train', data=train[0], dtype='int8', compression="gzip")
    Y_train = fout.create_dataset('Y_train', data=train[1], dtype='int8', compression="gzip")
    X_valid = fout.create_dataset('X_valid', data=valid[0], dtype='int8', compression="gzip")
    Y_valid = fout.create_dataset('Y_valid', data=valid[1], dtype='int8', compression="gzip")
    X_test = fout.create_dataset('X_test', data=test[0], dtype='int8', compression="gzip")
    Y_test = fout.create_dataset('Y_test', data=test[1], dtype='int8', compression="gzip")